In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy import displacy
import json
import random
import re

In [2]:
#Import original data
df_orginal = pd.read_excel('./data/qb/original/ios.xlsx')

In [3]:
df_orginal.head()

,App,Store,App ID,Review ID,Country,Version,Rating,Date,Author,Subject,...,Translated Subject,Translated Body,Sentiment,Device,Language,OS Version,Reply URL,Topics,Custom Topics,Tags
0,QuickBooks Accounting,iOS,584606479,5.467988e+09,USA,20.01.6,1,2020-01-31,NanaHermanson,Shiny useless tool,...,NaN,NaN,negative,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Advertising"", ""Bugs"", ""Design & UX"", ""Dissat...","[""Ease of Use""]",[]
1,QuickBooks Accounting,iOS,584606479,5.464235e+09,Canada,20.01.5,1,2020-01-30,Tugboat55,Latest app update,...,NaN,NaN,negative,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Bugs"", ""Feature Requests"", ""Dissatisfied use...",[],[]
2,QuickBooks Accounting,iOS,584606479,5.467098e+09,USA,20.01.5,3,2020-01-30,scholarbev,App closes out while entering expenses,...,NaN,NaN,negative,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Bugs"", ""Frequency"", ""Pricing"", ""Customer Sup...","[""FDX Filter""]",[]
3,QuickBooks Accounting,iOS,584606479,5.466985e+09,USA,20.01.5,5,2020-01-30,SOSA EL DINERO,Life saver,...,NaN,NaN,positive,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Design & UX"", ""Performance"", ""Satisfied user...",[],[]
4,QuickBooks Accounting,iOS,584606479,5.467691e+09,USA,20.01.6,5,2020-01-30,Sra. Linda,Small business,...,NaN,NaN,positive,NaN,English,NaN,https://app.appbot.co/apps/311455-quickbooks-a...,"[""Satisfied users""]",[],[]


In [4]:
#Get selective columns

df = df_orginal[['Review ID','Country', 'Version', 'Rating', 'Date', 'Subject', 'Body', 'Sentiment']]

In [5]:
df.head()

,Review ID,Country,Version,Rating,Date,Subject,Body,Sentiment
0,5.467988e+09,USA,20.01.6,1,2020-01-31,Shiny useless tool,This app looks awesome. Very well thought out...,negative
1,5.464235e+09,Canada,20.01.5,1,2020-01-30,Latest app update,This update is terrible basically unusable ple...,negative
2,5.467098e+09,USA,20.01.5,3,2020-01-30,App closes out while entering expenses,The majority of the time this app is wonderful...,negative
3,5.466985e+09,USA,20.01.5,5,2020-01-30,Life saver,One my favorite apps to use while working. I’m...,positive
4,5.467691e+09,USA,20.01.6,5,2020-01-30,Small business,This absolutely works perfectly without diffic...,positive


In [6]:
df['Subject'] = df['Subject'].replace(np.nan, '', regex=True)
df['Body'] = df['Body'].replace(np.nan, '', regex=True)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
df.head()

,Review ID,Country,Version,Rating,Date,Subject,Body,Sentiment
0,5.467988e+09,USA,20.01.6,1,2020-01-31,Shiny useless tool,This app looks awesome. Very well thought out...,negative
1,5.464235e+09,Canada,20.01.5,1,2020-01-30,Latest app update,This update is terrible basically unusable ple...,negative
2,5.467098e+09,USA,20.01.5,3,2020-01-30,App closes out while entering expenses,The majority of the time this app is wonderful...,negative
3,5.466985e+09,USA,20.01.5,5,2020-01-30,Life saver,One my favorite apps to use while working. I’m...,positive
4,5.467691e+09,USA,20.01.6,5,2020-01-30,Small business,This absolutely works perfectly without diffic...,positive


In [8]:
#combine subject and body
df['Review'] = df[['Subject', 'Body']].apply(lambda x: '. '.join(x), axis=1)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
df.head()

,Review ID,Country,Version,Rating,Date,Subject,Body,Sentiment,Review
0,5.467988e+09,USA,20.01.6,1,2020-01-31,Shiny useless tool,This app looks awesome. Very well thought out...,negative,Shiny useless tool. This app looks awesome. V...
1,5.464235e+09,Canada,20.01.5,1,2020-01-30,Latest app update,This update is terrible basically unusable ple...,negative,Latest app update. This update is terrible bas...
2,5.467098e+09,USA,20.01.5,3,2020-01-30,App closes out while entering expenses,The majority of the time this app is wonderful...,negative,App closes out while entering expenses. The ma...
3,5.466985e+09,USA,20.01.5,5,2020-01-30,Life saver,One my favorite apps to use while working. I’m...,positive,Life saver. One my favorite apps to use while ...
4,5.467691e+09,USA,20.01.6,5,2020-01-30,Small business,This absolutely works perfectly without diffic...,positive,Small business. This absolutely works perfectl...


In [10]:
#Drop Subject and Body

del df['Subject']
del df['Body']

In [11]:
df.head()

,Review ID,Country,Version,Rating,Date,Sentiment,Review
0,5.467988e+09,USA,20.01.6,1,2020-01-31,negative,Shiny useless tool. This app looks awesome. V...
1,5.464235e+09,Canada,20.01.5,1,2020-01-30,negative,Latest app update. This update is terrible bas...
2,5.467098e+09,USA,20.01.5,3,2020-01-30,negative,App closes out while entering expenses. The ma...
3,5.466985e+09,USA,20.01.5,5,2020-01-30,positive,Life saver. One my favorite apps to use while ...
4,5.467691e+09,USA,20.01.6,5,2020-01-30,positive,Small business. This absolutely works perfectl...


In [12]:
filename = "./data/qb/processed/train_ner.json"
print(filename)


with open(filename) as train_data:
    train = json.load(train_data)

TRAIN_DATA_NEW = []
for data in train:
    ents = [tuple(entity) for entity in data['entities']]
    TRAIN_DATA_NEW.append((data['content'],{'entities':ents}))


with open('{}'.format(filename.replace('json','txt')),'w') as write:
    write.write(str(TRAIN_DATA_NEW))


./data/qb/processed/train_ner.json


In [13]:

def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.load('en_core_web_sm')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe("ner")
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp


prdnlp = train_spacy(TRAIN_DATA_NEW, 20)

Statring iteration 0
{'ner': 1986.9761505657932}
Statring iteration 1
{'ner': 2033.6251475981499}
Statring iteration 2
{'ner': 1921.8103149888077}
Statring iteration 3
{'ner': 2025.8895919006648}
Statring iteration 4
{'ner': 1909.3573122694534}
Statring iteration 5
{'ner': 1944.9211737362523}
Statring iteration 6
{'ner': 1894.8798286345425}
Statring iteration 7
{'ner': 1843.8758145522281}
Statring iteration 8
{'ner': 1869.8035415216254}
Statring iteration 9
{'ner': 1871.5894217215991}
Statring iteration 10
{'ner': 1806.3899819964288}
Statring iteration 11
{'ner': 1821.4295198881468}
Statring iteration 12
{'ner': 1826.9713878296993}
Statring iteration 13
{'ner': 1911.349537615699}
Statring iteration 14
{'ner': 1877.4315822365943}
Statring iteration 15
{'ner': 1865.6955113855215}
Statring iteration 16
{'ner': 1963.5822984827582}
Statring iteration 17
{'ner': 1945.2511099126673}
Statring iteration 18
{'ner': 1733.9761223010664}
Statring iteration 19
{'ner': 1712.2175214665594}


In [14]:
# Save our trained Model
prdnlp.to_disk("./model/qbo_aspect")

In [33]:
#Test your text
test_text = ["Cancelling your subscription seems impossible. I've emailed, called, used their help menu. It shouldn't be this hard. Still trying to cancel.",
              "My App has stopped working on my Samsung s9 for 2 days in a row. You should have sent an email to inform subscribers if your app is going to be experiencing a downtime.",
              "Cannot open customer invoices. Please correct ASAP!",
               "bugs on iOS13 please update ASAP!"]
for text in test_text:
    print('-----')
    doc = prdnlp(text)
    print(doc)
    for ent in doc.ents:
        print('keyword=' + ent.text, 'aspect=' + ent.label_)
        ent_str = str(ent)
        print(ent_str.find("update"))

-----
Cancelling your subscription seems impossible. I've emailed, called, used their help menu. It shouldn't be this hard. Still trying to cancel.
keyword=subscription aspect=Subscription
-1
keyword=called aspect=Quality
-1
-----
My App has stopped working on my Samsung s9 for 2 days in a row. You should have sent an email to inform subscribers if your app is going to be experiencing a downtime.
keyword=App aspect=App
-1
-----
Cannot open customer invoices. Please correct ASAP!
keyword=invoices aspect=Invoicing
-1
-----
bugs on iOS13 please update ASAP!
keyword=bugs aspect=Quality
-1
keyword=update aspect=Updates
0


In [27]:
spacy.displacy.render(doc, style='ent', jupyter=True)

In [34]:
for index, row in df.iterrows():
    aspect_terms = []
    aspect_keywords = []
    doc = prdnlp(row['Review'])
    for ent in doc.ents:
        if ent.label_ not in aspect_terms:
            aspect_terms.append(ent.label_)
            aspect_keywords.append(ent.text)


    df.loc[index,'Aspects'] = ", ".join(aspect_terms)
    df.loc[index,'Aspects_Keywords'] = ", ".join(aspect_keywords)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [35]:
df.head()

,Review ID,Country,Version,Rating,Date,Sentiment,Review,Aspects,Aspects_Keywords
0,5.467988e+09,USA,20.01.6,1,2020-01-31,negative,Shiny useless tool. This app looks awesome. V...,"App, Expenses, Customer Service","app, expenses, support"
1,5.464235e+09,Canada,20.01.5,1,2020-01-30,negative,Latest app update. This update is terrible bas...,"App, Updates","app, update"
2,5.467098e+09,USA,20.01.5,3,2020-01-30,negative,App closes out while entering expenses. The ma...,"App, Expenses, Quality, Subscription","App, expenses, issue, subscription"
3,5.466985e+09,USA,20.01.5,5,2020-01-30,positive,Life saver. One my favorite apps to use while ...,Dashboard,One
5,5.460553e+09,USA,20.01.4,1,2020-01-29,negative,Worst update yet. Just deleted. Crashed. Didn’...,"Updates, Quality","update, deleted"


In [37]:
#number of reviews without aspect
(df['Aspects'].values == '').sum() 

0

In [38]:
len(df)

6085

In [39]:
df['Aspects'].replace('', np.nan, inplace=True)
df.dropna(subset=['Aspects'], inplace=True)

/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/eelrufaie/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
len(df)

6085

In [41]:
df.head()

,Review ID,Country,Version,Rating,Date,Sentiment,Review,Aspects,Aspects_Keywords
0,5.467988e+09,USA,20.01.6,1,2020-01-31,negative,Shiny useless tool. This app looks awesome. V...,"App, Expenses, Customer Service","app, expenses, support"
1,5.464235e+09,Canada,20.01.5,1,2020-01-30,negative,Latest app update. This update is terrible bas...,"App, Updates","app, update"
2,5.467098e+09,USA,20.01.5,3,2020-01-30,negative,App closes out while entering expenses. The ma...,"App, Expenses, Quality, Subscription","App, expenses, issue, subscription"
3,5.466985e+09,USA,20.01.5,5,2020-01-30,positive,Life saver. One my favorite apps to use while ...,Dashboard,One
5,5.460553e+09,USA,20.01.4,1,2020-01-29,negative,Worst update yet. Just deleted. Crashed. Didn’...,"Updates, Quality","update, deleted"


In [42]:
#generate aspect position index

df_split_terms = pd.DataFrame(columns=['Review ID', 'Country', 'Version', 'Rating', 'Date', 'Sentiment', 'Review', 'Aspects', 'Positions'])


for index, row in df.iterrows():
    aspects = [x.strip() for x in row['Aspects_Keywords'].lower().split(',')]
    
    positions = ''
    for asp in aspects:
        
        review = " ".join(row['Review'].lower().split())
        
        start_idx = review.find(asp)
        
        #if the aspect is not found in the review, skip
        if start_idx == -1:
            continue
            
        sentiment = 'neutral' if row['Sentiment'] == 'mixed' else row['Sentiment']
        
        df_split_terms = df_split_terms.append({
            'Review ID': row['Review ID'], 
            'Country': row['Country'], 
            'Version': row['Version'],
            'Rating': row['Rating'],
            'Date': row['Date'],
            'Sentiment': sentiment,
            'Review': review,
            'Aspects': asp,
            'Positions': str(start_idx) + ',' + str(start_idx + len(asp) - 1),
        }, ignore_index=True)
        
        #if positions == '':
        #    positions = str(start_idx) + '-' + str(start_idx + len(asp))
        #else:
        #    positions += ',' + str(start_idx) + '-' + str(start_idx + len(asp))
    
    #df.loc[index,'Positions'] = positions

In [24]:
df_split_terms.head()

,Review ID,Country,Version,Rating,Date,Sentiment,Review,Aspects,Positions
0,5.467988e+09,USA,20.01.6,1,2020-01-31,negative,shiny useless tool. this app looks awesome. ve...,app,"25,27"
1,5.467988e+09,USA,20.01.6,1,2020-01-31,negative,shiny useless tool. this app looks awesome. ve...,expenses,"190,197"
2,5.464235e+09,Canada,20.01.5,1,2020-01-30,negative,latest app update. this update is terrible bas...,app,"7,9"
3,5.467098e+09,USA,20.01.5,3,2020-01-30,negative,app closes out while entering expenses. the ma...,app,"0,2"
4,5.467098e+09,USA,20.01.5,3,2020-01-30,negative,app closes out while entering expenses. the ma...,expenses,"30,37"


In [43]:
df_split_terms.head()

,Review ID,Country,Version,Rating,Date,Sentiment,Review,Aspects,Positions
0,5.467988e+09,USA,20.01.6,1,2020-01-31,negative,shiny useless tool. this app looks awesome. ve...,app,"25,27"
1,5.467988e+09,USA,20.01.6,1,2020-01-31,negative,shiny useless tool. this app looks awesome. ve...,expenses,"190,197"
2,5.467988e+09,USA,20.01.6,1,2020-01-31,negative,shiny useless tool. this app looks awesome. ve...,support,"435,441"
3,5.464235e+09,Canada,20.01.5,1,2020-01-30,negative,latest app update. this update is terrible bas...,app,"7,9"
4,5.464235e+09,Canada,20.01.5,1,2020-01-30,negative,latest app update. this update is terrible bas...,update,"11,16"


In [83]:
#split data in - train test and dev
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    
    perm = np.random.permutation(np.arange(len(df)))
    m = len(df)
    
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end - 1] - 1]
    validate = df.iloc[perm[train_end:validate_end - 1] - 1]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [84]:
train, validate, test = train_validate_test_split(df_split_terms)

In [85]:
train.head()

,Review ID,Country,Version,Rating,Date,Sentiment,Review,Aspects,Positions
1934,1.566345e+09,USA,5.2.1,1,2017-03-17,negative,not reliable. is not reliable. not a replaceme...,app,"61,63"
3459,1.288820e+09,USA,4.3.3,4,2015-11-19,neutral,"awesome, but somewhat lacking. i'm an it profe...",customer,"1024,1031"
2895,1.421832e+09,USA,4.4.3,4,2016-07-28,positive,small business. i use it for estimates and inv...,estimates,"29,37"
2091,1.540071e+09,USA,5.2,5,2017-02-08,positive,love it!!!. this app makes things so easy to k...,app,"17,19"
2383,1.488553e+09,USA,5.0.1,5,2016-11-21,positive,owner. app is a great convenience,app,"7,9"


In [86]:
len(train)

2630

### Generate Training files

In [87]:
#save reviews
df_review = train[['Review']]
df_review.to_csv('review.txt', header=None, index=None, sep='\n', mode='a')

#save aspects
df_review = train[['Aspects']]
df_review.to_csv('term.txt', header=None, index=None, sep='\n', mode='a')

#save positions
df_review = train[['Positions']]
df_review.to_csv('position.txt', header=None, index=None, sep='\n', mode='a')

#save label
df_review = train[['Sentiment']]
df_review.to_csv('label.txt', header=None, index=None, sep='\n', mode='a')

### Generate Test files

In [88]:
#save reviews
df_review = test[['Review']]
df_review.to_csv('review.txt', header=None, index=None, sep='\n', mode='a')

#save aspects
df_review = test[['Aspects']]
df_review.to_csv('term.txt', header=None, index=None, sep='\n', mode='a')

#save positions
df_review = test[['Positions']]
df_review.to_csv('position.txt', header=None, index=None, sep='\n', mode='a')

#save label
df_review = test[['Sentiment']]
df_review.to_csv('label.txt', header=None, index=None, sep='\n', mode='a')

### Generate Dev files

In [89]:
#save reviews
df_review = validate[['Review']]
df_review.to_csv('review.txt', header=None, index=None, sep='\n', mode='a')

#save aspects
df_review = validate[['Aspects']]
df_review.to_csv('term.txt', header=None, index=None, sep='\n', mode='a')

#save positions
df_review = validate[['Positions']]
df_review.to_csv('position.txt', header=None, index=None, sep='\n', mode='a')

#save label
df_review = validate[['Sentiment']]
df_review.to_csv('label.txt', header=None, index=None, sep='\n', mode='a')